# Decomposing the Capacitated p-Median Problem
The capacitated p-median problem (CPMP) is a known and well-studied problem from the literature. Given
$n \in \mathbb{N}$ locations, the task is to select $p \in \mathbb{N}$ *median* locations with $p \leq n$ and to
assign each location to exactly one median. For any two locations $i,j \in [n]$, the distance between them is given
by $d_{ij} \in \mathbb{R}$. The distance between the locations and their assigned medians is minimized. Every
location $i \in [n]$ has a *demand* $q_i \in \mathbb{R}$ and a maximum *capacity* $Q_i \in \mathbb{R}$. For every
selected median, the sum of the demands of the locations assigned to it must not exceed its capacity.

The CPMP can be formulated as a MIP. Here is a classical compact formulation of problem:

$$
\begin{alignat*}{3}
z^{\star}_{IP}\,\,\,=\,\,\,\min{} &\sum_{i=1}^n \sum_{j=1}^n d_{ij} x_{ij} \hspace{-2em} &&&&\\
                     \text{s. t.} &\sum_{j=1}^n  x_{ij} &&= 1 \quad &\forall i &\in [n]\\
                                  &\sum_{i=1}^{n} q_i x_{ij} &&\leq Q_j y_j \quad &\forall j &\in [n] \\
                                  &\sum_{j=1}^n  y_j &&= p && \\
                                  &x_{ij} \in \{0,1\}, y_j \in \{0,1\} \hspace{-6em} &&\hspace{6em}\quad& \forall i,j &\in [n].
\end{alignat*}
$$

There are different approaches to solve the CPMP. As it has a structure, we can try to solve using a
Branch-Cut-and-Price approach. For that we want to use the `PyGCGOpt` interface to interact with GCG. We will consider three
use-cases: (1) The Automatic Mode, (2) Exploring different Decompositions, and (3) Building a custom Decomposition.

To follow along with this tutorial interactively, please download the Jupyter notebook from the [examples folder](https://github.com/scipopt/PyGCGOpt/tree/master/examples/cpmp).

## Reading in the Instance
The `PyGCGOpt` interface offers two methods to specify a problem. The first is to load the model from a standardized file format (e.g., `.lp` or `.mps`) that is supported by `SCIP` using `Model.readProb()`. Optionally, one can also read in a decomposition from a `.dec` file in the same manner. In this example, we will use the second method: Modeling a problem directly in Python.

Execute the following cell which includes a trivial test instance and function to load more instances from a custom `JSON` file format.

In [1]:
import json
import pygcgopt
print(pygcgopt.__version__)
def get_simple_instance():
    n = 5
    p = 2
    d = {0: {0: 0, 1: 25, 2: 46, 3: 43, 4: 30}, 1: {1: 0, 2: 22, 3: 20, 4: 22}, 2: {2: 0, 3: 22, 4: 40}, 3: {3: 0, 4: 22}, 4: {4: 0}}
    q = {0: 14, 1: 13, 2: 9, 3: 15, 4: 6}
    Q = {i: 33 for i in range(5)}
    return n, p, d, q, Q

def read_instance_json(path):
    with open(path) as f:
        instance = json.load(f)
    d = {int(k): {int(kk): vv for kk, vv in v.items()} for k, v in instance["d"].items()}
    q = {int(k): v for k, v in instance["q"].items()}
    Q = {int(k): v for k, v in instance["Q"].items()}
    return instance["n"], instance["p"], d, q, Q

n_locations, n_clusters, distances, demands, capacities = read_instance_json("instances/p550-01.json")

0.4.0


## Setting up the Model
Now, we want to build the model based on the above formulation. Please note that this part is *not* specific to GCG but is identical to how one would build the same model with `PySCIPOpt`. In technical terms, `Model` in `PyGCGOpt` is a subclass of `Model`  in `PySCIPOpt` and, therefore, you can use all methods of `PySCIPOpt` `Model` to build your problem.

In order to recreate the model multiple times during this example, we create a method that will return the model. The method also returns the different constraints added to the model grouped by type. This will be important later in use-case 3.

In [2]:
from pygcgopt import Model, quicksum


def build_model(n_locations, n_clusters, distances, demands, capacities):
    m = Model()

    m.printVersion()
    m.redirectOutput()

    m.setMinimize()

    x = {}
    y = {}

    for j in range(n_locations):
        y[j] = m.addVar(f"y_{j}", vtype="B")
        for i in range(n_locations):
            x[i, j] = m.addVar(f"x_{i}_{j}", vtype="B", obj=distances[min(i,j)][max(i,j)])

    # Hold different constraint types
    conss_assignment = []
    conss_capacity = []
    cons_pmedian = None

    # Create the assignment constraints
    for i in range(n_locations):
        conss_assignment.append(
            m.addCons(quicksum(x[i, j] for j in range(n_locations)) == 1)
        )

    # Create the capacity constraints
    for j in range(n_locations):
        conss_capacity.append(
            m.addCons(quicksum(demands[i] * x[i, j] for i in range(n_locations)) <= capacities[j] * y[j])
        )

    # Create the p-median constraint
    cons_pmedian = m.addCons(quicksum(y[j] for j in range(n_locations)) == n_clusters)

    return m, conss_assignment, conss_capacity, cons_pmedian

## Use-Case 1: The Automatic Mode
With the model built, we can now simply call the `optimize()` function and let GCG do its magic.

In [3]:
m, *conss = build_model(n_locations, n_clusters, distances, demands, capacities)
m.optimize()

GCG version 3.6.2 [GitHash: NoGitInfo]
Copyright (C) 2010-2024 Operations Research, RWTH Aachen University
                        Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

SCIP version 9.1.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.1.1] [GitHash: NoGitInfo]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 100 upgd conss, 0 impls, 50 clqs
   (0.0s) probing: 51/2550 (2.0%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 2550 cliques
presolved problem has 2550 variables (2550 bin, 0 int, 0 impl, 0 cont) and 101 constraints
     50 constrai

## Use-Case 2: Exploring different Decompositions
Above, we have seen GCG in its fully automatic mode. If we want to dig deeper, we can inspect the different decompositions that GCG detects. For that, we recreate the model and manually execute `presolve()` and `detect()` for the model. At this stage it is possible to list and visualize the found decompositions.

In [4]:
m, *conss = build_model(n_locations, n_clusters, distances, demands, capacities)
m.presolve()
m.detect()

decomps = m.listDecompositions()

print("GCG found {} finnished decompositions.".format(len(decomps)))

GCG version 3.6.2 [GitHash: NoGitInfo]
Copyright (C) 2010-2024 Operations Research, RWTH Aachen University
                        Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

SCIP version 9.1.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.1.1] [GitHash: NoGitInfo]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 100 upgd conss, 0 impls, 50 clqs
   (0.0s) probing: 51/2550 (2.0%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 2550 cliques
presolved problem has 2550 variables (2550 bin, 0 int, 0 impl, 0 cont) and 101 constraints
     50 constrai

### Inspecting Decompositions

The call to `listDecompositions()` returns a list of `PartialDecomposition` objects. We can print a decomposition using the Python `print()` function to get a summary or access different fields directly.

For a full overview of available methods, take a look at the online documentation for the `PartialDecomposition` class, or execute `help(d)` where `d` is a decomposition object.

In [5]:
# Instead of print(decomps), let's print each decomposition's scores explicitly
print(f"\nNumber of decompositions: {len(decomps)}")
for i, dec in enumerate(decomps):
    print(f"\nDecomposition {i} scores:")
    print(f"Classic score: {dec.classic_score:.04f}")
    print(f"Border area score: {dec.border_area_score:.04f}")
    print(f"Max white score: {dec.max_white_score:.04f}")
    print(f"Max for white score: {dec.max_for_white_score:.04f}")

# Then continue with selecting decomposition 2
d = decomps[2]
print(
    f"\nSelected decomposition scores: {d.classic_score=:.04f}, {d.border_area_score=:.04f}, {d.max_white_score=:.04f}, {d.max_for_white_score=:.04f}"
)


Number of decompositions: 11

Decomposition 0 scores:
Classic score: 1.0000
Border area score: 0.0000
Max white score: 0.0000
Max for white score: 0.0000

Decomposition 1 scores:
Classic score: 0.7040
Border area score: 1.0000
Max white score: 0.0000
Max for white score: 0.0000

Decomposition 2 scores:
Classic score: 0.5970
Border area score: 0.4950
Max white score: 0.4851
Max for white score: 0.4851

Decomposition 3 scores:
Classic score: 0.6030
Border area score: 0.5050
Max white score: 0.4950
Max for white score: 0.4950

Decomposition 4 scores:
Classic score: 0.4070
Border area score: 0.5050
Max white score: 0.0000
Max for white score: 0.0000

Decomposition 5 scores:
Classic score: 0.3001
Border area score: 0.0099
Max white score: 0.0097
Max for white score: 0.0097

Decomposition 6 scores:
Classic score: 0.6980
Border area score: 0.9901
Max white score: 0.0000
Max for white score: 0.0000

Decomposition 7 scores:
Classic score: 0.5912
Border area score: 0.4950
Max white score: 0.485

### Visualizing Decompositions
In addition, GCG can create graphical visualizations of decompositions. They can easily be displayed in a Jupyter nodebook like so:

In [ ]:
d

AttributeError: 'pygcgopt.gcg.PartialDecomposition' object has no attribute 'maxForWhiteScore'

sh: 1: gnuplot: not found


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmplxh7_wqc/vis.svg'

sh: 1: gnuplot: not found


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmp8y_rajlu/vis.png'

In [ ]:
# Replace the simple "d" line with:
print("Decomposition visualization:")
print(f"Number of blocks: {d.getNBlocks()}")
print(f"Number of master constraints: {d.getNMasterconss()}")
print(f"Number of linking variables: {d.getNLinkingvars()}")
print(f"Scores:")
print(f"  Classic score: {d.classic_score:.04f}")
print(f"  Border area score: {d.border_area_score:.04f}")
print(f"  Max white score: {d.max_white_score:.04f}")
print(f"  Max for white score: {d.max_for_white_score:.04f}")

# Show block information
for block in range(d.getNBlocks()):
    print(f"\nBlock {block}:")
    print(f"  Number of constraints: {d.getNConssForBlock(block)}")
    print(f"  Number of variables: {d.getNVarsForBlock(block)}")

### Selecting Decompositions
After this investigation, we decide that we like this particular decomposition. The following code orders GCG to select our decomposition instead of an automatic one. Then, the optimization process is started.

In [8]:
d.isSelected = True

m.optimize()


A Dantzig-Wolfe reformulation is applied to solve the original problem.
Chosen structure has 50 blocks and 51 linking constraints.
This decomposition has a maxwhite score of 0.485149.
Matrix has 50 blocks, using 50 pricing problems.

  time | node  | left  |SLP iter|MLP iter|LP it/n| mem |mdpt |ovars|mvars|ocons|mcons|mcuts|  dualbound   | primalbound  |  deg   |  gap   
p  0.2s|     1 |     0 |      0 |      0 |     - |  36M|   0 |2550 |   0 | 102 |   0 |   0 | 0.000000e+00 | 2.711000e+03 |   --   |    Inf 
p  0.2s|     1 |     0 |      0 |      0 |     - |  36M|   0 |2550 |   0 | 102 |   0 |   0 | 0.000000e+00 | 1.785000e+03 |   --   |    Inf 
p  0.2s|     1 |     0 |      0 |      0 |     - |  36M|   0 |2550 |   0 | 102 |   0 |   0 | 0.000000e+00 | 1.102000e+03 |   --   |    Inf 
   0.2s|     1 |     0 |      0 |      0 |     - |  36M|   0 |2550 |   0 | 102 |   0 |   0 | 0.000000e+00 | 1.102000e+03 |   --   |    Inf 

     
   0.2s|     1 |     0 |      0 |      0 |     - |  36M|  

## Use-Case 3: Building a custom Decomposition
In the previous use-cases we run GCG with automatically detected decompositions. This is useful if we do not know the exact structure of a model but still want to exploit GCG's decomposition approach.

However, for our model we *do* know its structure. If you take another look at our `build_model()` method, you will notice that we store the created constraints in different variables based on their type. This is a typical approach when we want to specify a custom decomposition after building the model using the Python interface.

In the following code, we recreate our model and use the different constraint types fo select constraints for reformulation and constraints for the Dantzig-Wolfe master problem.

In [10]:
m, conss_assignment, conss_capacity, cons_pmedian = build_model(
    n_locations, n_clusters, distances, demands, capacities
)

conss_master = conss_assignment + [cons_pmedian]
conss_reform = conss_capacity

pd = m.createPartialDecomposition()
pd.fixConssToMaster(conss_master)

for block, c in enumerate(conss_reform):
    pd.fixConsToBlock(c, block)

# m.addPreexistingPartialDecomposition(pd)

m.optimize()

GCG version 3.6.2 [GitHash: NoGitInfo]
Copyright (C) 2010-2024 Operations Research, RWTH Aachen University
                        Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

SCIP version 9.1.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.1.1] [GitHash: NoGitInfo]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)


AttributeError: 'pygcgopt.gcg.Model' object has no attribute 'createPartialDecomposition'

In [ ]:
m, conss_assignment, conss_capacity, cons_pmedian = build_model(
    n_locations, n_clusters, distances, demands, capacities
)

try:
    pd = m.createPartialDecomposition()
    
    # Use fixConssToMaster instead of addConsToMaster
    pd.fixConssToMaster(conss_assignment + [cons_pmedian])
    
    # Use fixConsToBlock instead of addConsToBlock
    for block, cons in enumerate(conss_capacity):
        pd.fixConsToBlock(cons, block)
    
    # Set decomposition
    m.addPreexistingPartialDecomposition(pd)
    m.optimize()
    
except Exception as e:
    print(f"Error in custom decomposition: {str(e)}")
    print("Falling back to automatic decomposition")
    m.optimize()

## Summary

With that, we have seen the most important features to use GCG as a solver through the Python interface. In a different example, we will take a look at how GCG can be extended using Python code.